In [1]:

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import matplotlib.pyplot as plt

In [2]:
# URL de los datos oficiales que se van a trabajar
#Datos oficiales
url_prod_2017 = "Crudo 2017.xlsx"
url_prod_2018 = "Crudo 2018.xlsx"
url_prod_2019 = "Crudo 2019-DIC.xlsx"

#Datos blindtest
url_prod_2017bt = "/content/Producción 2017.xlsx"
url_prod_2018bt = "Producción bt 2018.xlsx"
url_prod_2019bt = "/content/Producción bt 2019.xlsx"


In [3]:
# Funcion que descarga, almaceno y limpia los datos
def get_data(data, year):
    num=None
    from_url =  pd.read_excel(data,header=None)                  # Carga el archivo de excel a una variable temporal 
    for i in range(len(from_url)):                   # Encuentra los nombres de las columnas y reasigna el encabezado
        if from_url.iloc[i,0] == "Departamento":          # A partir de la palabra "Departamento" en la primera columna 
            num = i
            break
    new_df = from_url.iloc[num:].reset_index(drop=True)
    encabezado = new_df.iloc[0].str.capitalize()     # Estandariza los nombres del encabezado
    new_df = new_df.rename(columns=encabezado)
    new_df = new_df.iloc[1:].reset_index(drop=True)
    new_df['Diciembre']=new_df['Diciembre'].replace(np.nan,0)
    new_df = new_df.dropna(subset=['Municipio']).dropna(axis=1)
    new_df["Año"] = year                             # Crea una nueva columna con el año al que corresponden los datos
    
    return new_df

In [4]:
# Obtiene los archivos y almacena la información en un data frame para cada año
#Datos oficiales
df_2017 = get_data (url_prod_2017, 2017)
df_2018 = get_data (url_prod_2018, 2018)
df_2019 = get_data (url_prod_2019, 2019)
#Datos Blindtest
df_2017bt = get_data (url_prod_2017bt, 2017)
df_2018bt = get_data (url_prod_2018bt, 2018)
df_2019bt = get_data (url_prod_2019bt, 2019)

df_2017
# Ejemplo

,Departamento,Municipio,Operadora,Contrato,Campo,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Año
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,1230.97,1108.93,1026.16,1035.77,1046.87,1122.08,1133.03,1163.72,1130.25,1077.69,1156.24,1099.277742,2017
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,NARE,271.742,257.036,252.903,232.9,261.774,216.059,193.717,161.835,231.971,244.219,222.163,208.824516,2017
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,UNDERRIVER,935.677,796.929,880.226,899.867,993.742,801.946,765.493,814.503,887.706,845.619,720.345,833.261290,2017
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,393,386,361.613,365.433,395.613,381.826,369.09,342.075,279.994,263.75,281.705,277.985484,2017
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,CASABE,CASABE,12745,12535.2,12756.4,12720.3,12491.9,13126.9,12376.6,12083.6,11965.4,11750.8,11931.9,11633.833226,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,TOLIMA,PRADO,HOCOL S.A.,ESPINAL - HOCOL,MATACHÍN SUR,310.129,286.929,260.065,213.7,182.806,219.567,125.29,90.8387,275.467,343.355,311.933,293.096774,2017
463,TOLIMA,PURIFICACIÓN,HOCOL S.A.,CHENCHE HOCOL,CHENCHE,59.5484,55.0357,53.6774,51.9667,51.2903,50.1,49.9032,50.7742,49.8667,46.3871,47.1333,50.225806,2017
464,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL - HOCOL,MATACHÍN NORTE,2219.45,2201.07,2156.74,2171.57,2131.97,2082.5,2081.16,2091.26,1980.97,1930.65,1799.9,1803.014839,2017
465,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL - HOCOL,MATACHÍN SUR,359.323,348.536,336.871,309.4,298.097,271.7,268.774,422.161,250.733,237.065,245.3,235.838710,2017


In [5]:
np.nan==None

False

In [6]:
# Guarda los archivos en un solo data frame para facilidad de manejo

base_datos = pd.concat([df_2017,df_2018,df_2019]).reset_index(drop=True)
base_datosbt = pd.concat([df_2017bt,df_2018bt,df_2019bt]).reset_index(drop=True)

base_datos = base_datos.infer_objects()
base_datosbt = base_datosbt.infer_objects()

base_datosbt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Departamento  122 non-null    object 
 1   Municipio     122 non-null    object 
 2   Operadora     122 non-null    object 
 3   Contrato      122 non-null    object 
 4   Campo         122 non-null    object 
 5   Enero         122 non-null    float64
 6   Febrero       122 non-null    float64
 7   Marzo         122 non-null    float64
 8   Abril         122 non-null    float64
 9   Mayo          122 non-null    float64
 10  Junio         122 non-null    float64
 11  Julio         122 non-null    float64
 12  Agosto        122 non-null    float64
 13  Septiembre    122 non-null    float64
 14  Octubre       122 non-null    float64
 15  Noviembre     122 non-null    float64
 16  Diciembre     122 non-null    float64
 17  Año           122 non-null    int64  
dtypes: float64(12), int64(1), obje

In [7]:
# Funcion que transforma la data en formato LONG

def long(data):
    melted = pd.melt(data, id_vars=["Departamento","Municipio","Operadora", "Contrato", "Campo","Año"],\
                    value_vars = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre",\
                                  "Octubre","Noviembre","Diciembre"],\
                    var_name = "Mes",\
                    value_name ="Produccion")
    return melted

In [8]:
# Nuevos datos en formato LONG, que será nuestra tabla de la base de datos

base_datos = long(base_datos)
base_datosbt = long(base_datosbt)

base_datos                  

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
0,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2017,Enero,1230.967742
1,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,NARE,2017,Enero,271.741935
2,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE ASOCIACION,UNDERRIVER,2017,Enero,935.677419
3,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,OPERACION-DIRECTA ECOPETROL,AREA TECA-COCORNA,2017,Enero,393.000000
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,CASABE,CASABE,2017,Enero,12745.000000
...,...,...,...,...,...,...,...,...
16699,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,MATACHÍN NORTE,2019,Diciembre,1234.760000
16700,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,PURIFICACIÓN,2019,Diciembre,55.380000
16701,TOLIMA,PURIFICACIÓN,HOCOL S.A.,ESPINAL,MATACHÍN SUR,2019,Diciembre,92.440000
16702,TOLIMA,PURIFICACIÓN,HOCOL S.A.,CHENCHE,CHENCHE,2019,Diciembre,51.130000


In [9]:
# Creación de la base de datos en SQLite

engine = create_engine("sqlite:///HackathonSPE.db")
sqlite_connection = engine.connect()

# Crea la tabla "TablaProduccion" en la base de datos, a partir de la información de la variable con los datos en formato long
#base_datos.to_sql("TablaProduccion", sqlite_connection, if_exists ="replace")     
base_datosbt.to_sql("TablaProduccionbt", sqlite_connection, if_exists ="replace") 

-----


**VERIFICACIÓN**

In [10]:
# Cargar datos a data frame a partir de la base de datos creada
# Para verificar que se haya creado correctamente la base de datos y que su contenido corresponda 


pd.options.display.float_format = '{:.2f}'.format       # Formato de dos decimales en floats

cnx = sqlite3.connect('HackathonSPE.db')
#df_produccion = pd.read_sql_query("SELECT * FROM TablaProduccion", cnx, index_col = "index")
df_produccionbt = pd.read_sql_query("SELECT * FROM TablaProduccionbt", cnx, index_col = "index")
#df_produccion

In [11]:
engine = create_engine("sqlite:///HackathonSPE.db")
sqlite_connection = engine.connect()

# Crea la tabla "TablaProduccion" en la base de datos, a partir de la información de la variable con los datos en formato long
base_datos.to_sql("TablaProduccion", sqlite_connection, if_exists ="replace")     
base_datosbt.to_sql("TablaProduccionbt", sqlite_connection, if_exists ="replace")

In [12]:
pd.options.display.float_format = '{:.2f}'.format       # Formato de dos decimales en floats

cnx = sqlite3.connect('HackathonSPE.db')
df_produccion = pd.read_sql_query("SELECT * FROM TablaProduccion", cnx, index_col = "index")
df_produccionbt = pd.read_sql_query("SELECT * FROM TablaProduccionbt", cnx, index_col = "index")


In [20]:
tasa=base_datosbt['Produccion'][76]/base_datos['Produccion'][944]
tasa

1.0874629461052692

In [22]:
def desencriptado(df, dfbt):
  for ind,prod in dfbt['Produccion'].iteritems():
    for ind2,prod2 in df['Produccion'].iteritems():
      if prod==round(prod2*(tasa),2): 
       dfbt['Departamento']=dfbt['Departamento'].replace(dfbt['Departamento'][ind],df['Departamento'][ind2])
       dfbt['Municipio']=dfbt['Municipio'].replace(dfbt['Municipio'][ind],df['Municipio'][ind2])
       dfbt['Operadora']=dfbt['Operadora'].replace(dfbt['Operadora'][ind],df['Operadora'][ind2])
       dfbt['Contrato']=dfbt['Contrato'].replace(dfbt['Contrato'][ind],df['Contrato'][ind2])
       dfbt['Campo']=dfbt['Campo'].replace(dfbt['Campo'][ind],df['Campo'][ind2])
       break
  
       
  
  return dfbt
desencriptado(base_datos,base_datosbt)

,Departamento,Municipio,Operadora,Contrato,Campo,Año,Mes,Produccion
0,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,CHIPIRÓN,2017,Enero,12371.08
1,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,GALEMBO,2017,Enero,5.37
2,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,MACANA,2017,Enero,120.46
3,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,GALEMBO,2017,Enero,23208.88
4,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,CAÑO YARUMAL,2017,Enero,4386.09
...,...,...,...,...,...,...,...,...
1459,PUTUMAYO,PUERTO ASIS,OCCIDENTAL DE COLOMBIA LLC,SURORIENTE,QUILLACINGA,2019,Diciembre,1084.29
1460,PUTUMAYO,PUERTO ASIS,OCCIDENTAL DE COLOMBIA LLC,SURORIENTE,QUINDE,2019,Diciembre,414.50
1461,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,GALEMBO,2019,Diciembre,0.00
1462,ARAUCA,ARAUQUITA,OCCIDENTAL DE COLOMBIA LLC,CHIPIRON,GALEMBO,2019,Diciembre,408.02
